# Evaluation of `passim` results

## Running the evaluation

In [1]:
import sys
sys.path.append('.')
from lib.evaluation import TextReuseEvaluator

In [2]:
GT_DATASET_PATH = 'data/homeric_repetitions_dataset.tsv'
PASSIM_OUTPUT_DIR = 'data/passim/exp5/out.json/'

evaluator = TextReuseEvaluator()

In [3]:
evaluator.read_predictions(PASSIM_OUTPUT_DIR)
evaluator.read_groundtruth(GT_DATASET_PATH)

41 predicted clusters found
69 groundtruth clusters found


In [4]:
evaluator.evaluate()

# of ground-truth cluster: 69
# of matched clusters: 7
# of unmatched clusters: 62


# of partially matched clusters: 1
# of exactly matched clusters: 4
# of clusters with spurious passages: 2


In [7]:
evaluator.matches

[<Match: GT cluster[1] with predicted cluster[1] (spurious-match)>,
 <Match: GT cluster[2] with predicted cluster[1] (spurious-match)>,
 <Match: GT cluster[15] with predicted cluster[35] (exact-match)>,
 <Match: GT cluster[22] with predicted cluster[8589934605] (exact-match)>,
 <Match: GT cluster[27] with predicted cluster[8589934612] (exact-match)>,
 <Match: GT cluster[28] with predicted cluster[8589934612] (partial-match)>,
 <Match: GT cluster[29] with predicted cluster[39] (exact-match)>]

In [8]:
for cluster in evaluator.gt_clusters.values():
    if cluster.matched_predictions:
        for match in cluster.matched_predictions:
            match.inspect()

GT cluster 1
	('<Locus: Homer, Iliad 2.11-2.15>') θωρῆξαί ἑ κέλευε κάρη κομόωντας Ἀχαιοὺς πανσυδίῃ· νῦν γάρ κεν ἕλοι πόλιν εὐρυάγυιαν Τρώων· οὐ γὰρ ἔτʼ ἀμφὶς Ὀλύμπια δώματʼ ἔχοντες ἀθάνατοι φράζονται· ἐπέγναμψεν γὰρ ἅπαντας Ἥρη λισσομένη, Τρώεσσι δὲ κήδεʼ ἐφῆπται.
	('<Locus: Homer, Iliad 2.28-2.32>') θωρῆξαί σε κέλευσε κάρη κομόωντας Ἀχαιοὺς πανσυδίῃ· νῦν γάρ κεν ἕλοις πόλιν εὐρυάγυιαν Τρώων· οὐ γὰρ ἔτʼ ἀμφὶς Ὀλύμπια δώματʼ ἔχοντες ἀθάνατοι φράζονται· ἐπέγναμψεν γὰρ ἅπαντας Ἥρη λισσομένη, Τρώεσσι δὲ κήδεʼ ἐφῆπται
	('<Locus: Homer, Iliad 2.66-2.69>') θωρῆξαί σε κέλευσε κάρη κομόωντας Ἀχαιοὺς πανσυδίῃ· νῦν γάρ κεν ἕλοις πόλιν εὐρυάγυιαν Τρώων· οὐ γὰρ ἔτʼ ἀμφὶς Ὀλύμπια δώματʼ ἔχοντες ἀθάνατοι φράζονται· ἐπέγναμψεν γὰρ ἅπαντας Ἥρη λισσομένη, Τρώεσσι δὲ κήδεʼ ἐφῆπται


Predicted cluster 1
	('<Locus: Homer, Iliad 2.23-2.34>') θωρέσ σύ κελεύω κάρα κομάω πανσυδία αἱρέω πόλις τρώς δῶμα ἔχω φράζω ἐπιγνάφω Ἥρα λίσσομαι Τρώς κῆδος φημί 
	('<Locus: Homer, Iliad 2.60-2.70>') θωρέσ σύ κελεύω κάρα κομ